# HW2: Problem 2: Working out Backpropagation

Read Chapter 2 of Michael Nielsen's article/book from top to bottom:

* [http://neuralnetworksanddeeplearning.com/chap2.html](http://neuralnetworksanddeeplearning.com/chap2.html)

He outlines a few exersizes in that article which you must complete. Do the following a, b, c:

a. He invites you to write out a proof of equation BP3

b. He invites you to write out a proof of equation BP4

c. He proposes that you code a fully matrix-based approach to backpropagation over a mini-batch. Implement this with explanation where you change the notation so that instead of having a bias term, you assume that the input variables are augmented with a "column" of "1"s, and that the weights $w_0$.

Your submission should be a single jupyter notebook. Use markdown cells with latex for equations of a jupyter notebook for each proof for "a." and "b.". Make sure you include text that explains your steps. Next for "c" this is an implementation problem. You need to understand and modify the code the Michael Nielsen provided so that instead it is using a matrixed based approach. Again don't keep the biases separate. After reading data in (use the iris data set), create a new column corresponding to $x_0=1$, and as mentioned above and discussed in class (see notes) is that the bias term can then be considered a weight $w_0$. Again use markdown cells around your code and comments to explain your work. Test the code on the iris data set with 4 node input (5 with a constant 1), three hidden nodes, and three output nodes, one for each species/class.

## a. Proof of Michael Nielsons equation BP3

Please fill this in using markdown and latex.

### Proof for BP3

The equation BP3 relates to the rate of change of the cost with respect to any weight in the network, given by:

$$
\frac{\partial C}{\partial w_{jk}^l} = a_k^{l-1} \delta_j^l
$$

#### Proof:

1. The cost $C$ is affected by the outputs of the network, which depend on the net inputs $z_j^l$ to the neurons in the final layer. The net input $z_j^l$ is influenced by the weights $w_{jk}^l$ and the activations $a_k^{l-1}$ from the previous layer.

2. The change in cost with respect to a change in a particular weight $w_{jk}^l$ is expressed using the chain rule:

$$
\frac{\partial C}{\partial w_{jk}^l} = \frac{\partial C}{\partial a_j^l} \cdot \frac{\partial a_j^l}{\partial z_j^l} \cdot \frac{\partial z_j^l}{\partial w_{jk}^l}
$$

3. The term $\frac{\partial C}{\partial a_j^l} \cdot \frac{\partial a_j^l}{\partial z_j^l}$ is defined as the error term $\delta_j^l$ for the neuron in layer $l$.

4. The term $\frac{\partial z_j^l}{\partial w_{jk}^l}$ is the activation $a_k^{l-1}$ from the previous layer, since $z_j^l = \sum_k w_{jk}^l a_k^{l-1} + b_j^l$.

5. Combining these, we get the desired expression:

$$
\frac{\partial C}{\partial w_{jk}^l} = a_k^{l-1} \delta_j^l
$$

This demonstrates that the gradient of the cost with respect to a weight is the product of the activation from the preceding layer and the error term from the current layer.


## b. Proof of Michael Nielsons equation BP4

Please fill this in using markdown and latex.

### Proof for BP4

The equation BP4 concerns the gradient of the cost function with respect to the biases in the network, given by:

$$
\frac{\partial C}{\partial b_j^l} = \delta_j^l
$$

#### Proof:

1. The bias $b_j^l$ affects the net input $z_j^l$ to the neuron, which then influences the neuron's activation $a_j^l$ and consequently the overall cost $C$.

2. Applying the chain rule, the rate of change of $C$ with respect to $b_j^l$ is expressed as:

$$
\frac{\partial C}{\partial b_j^l} = \frac{\partial C}{\partial a_j^l} \cdot \frac{\partial a_j^l}{\partial z_j^l} \cdot \frac{\partial z_j^l}{\partial b_j^l}
$$

3. By definition, $\frac{\partial z_j^l}{\partial b_j^l} = 1$ since $z_j^l = \sum_k w_{jk}^l a_k^{l-1} + b_j^l$, and the derivative of $z_j^l$ with respect to $b_j^l$ is 1.

4. The product $\frac{\partial C}{\partial a_j^l} \cdot \frac{\partial a_j^l}{\partial z_j^l}$ defines the error term $\delta_j^l$.

5. Thus, we obtain the equation:

$$
\frac{\partial C}{\partial b_j^l} = \delta_j^l
$$

This shows that the gradient of the cost with respect to a bias is simply the error term for the neuron.


## c. Using both markdown cells and code cells implement that you code a fully matrix-based approach to backpropagation over a mini-batch. Implement this with explanation where you change the notation so that instead of having a bias term, you assume that the input variables are augmented with a "column" of "1"s, and that the weights $w_0$.

In [1]:
# Code cell for part c.
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load the Iris dataset
iris = load_iris()
X = iris.data  # Input features
y = iris.target  # Target labels

# One-hot encode target labels
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Function to augment the input matrix with a column of ones for the bias term
def augment_input(X):
    return np.hstack([np.ones([X.shape[0], 1]), X])

X_train_aug = augment_input(X_train)
X_test_aug = augment_input(X_test)

# Initialize weight matrices
def initialize_weights(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size + 1, hidden_size)  # Including bias term in input size
    W2 = np.random.randn(hidden_size + 1, output_size)  # Including bias term in hidden size
    return W1, W2

input_size = X_train_aug.shape[1] - 1  # Exclude the bias term from input size
hidden_size = 3
output_size = y_train.shape[1]

W1, W2 = initialize_weights(input_size, hidden_size, output_size)

# Sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Forward pass through the network
def forward_pass(X, W1, W2):
    # Input to hidden layer
    Z1 = np.dot(X, W1)
    A1 = sigmoid(Z1)
    A1_aug = augment_input(A1)  # Augment for bias term in the hidden layer
    
    # Hidden to output layer
    Z2 = np.dot(A1_aug, W2)
    A2 = sigmoid(Z2)
    
    return A1, A2

# Example forward pass (remove in actual training loop)
A1, A2 = forward_pass(X_train_aug, W1, W2)



This step involves computing the gradients for the weight matrices. Since the focus here is on the implementation structure, I'll outline the gradient calculation without delving into the detailed math. The actual backpropagation computation depends on the loss function used and involves calculating the derivatives of the loss with respect to the weights, which can be complex.

In [9]:
def backpropagation(X_aug, Y, A1, A2, W1, W2):
    # Error in output
    dZ2 = A2 - Y
    A1_aug = augment_input(A1)  # Augment A1 with a column of ones for the bias term
    dW2 = np.dot(A1_aug.T, dZ2)

    # Error in hidden layer, excluding bias term from W2 for backpropagation
    dZ1 = np.dot(dZ2, W2[1:, :].T) * A1 * (1 - A1)
    dW1 = np.dot(X_aug.T, dZ1)

    return dW1, dW2


def update_weights(W1, W2, dW1, dW2, learning_rate):
    W1 -= learning_rate * dW1
    W2 -= learning_rate * dW2  # Ensure W2 and dW2 have compatible shapes
    return W1, W2

    # Cross-entropy loss function
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]  # Number of samples
    loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / m  # Add a small value to avoid log(0)
    return loss

# Training loop with loss computation
learning_rate = 0.01
epochs = 1000
N = 100  # Print the loss every 100 epochs

for epoch in range(epochs):
    # Forward pass
    A1, A2 = forward_pass(X_train_aug, W1, W2)
    
    # Compute gradients
    dW1, dW2 = backpropagation(X_train_aug, y_train, A1, A2, W1, W2)
    
    # Update weights
    W1, W2 = update_weights(W1, W2, dW1, dW2, learning_rate)
    
    # Compute and print loss every N epochs
    if epoch % N == 0:
        loss = cross_entropy_loss(y_train, A2)
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.9072
Epoch 100, Loss: 1.0983
Epoch 200, Loss: 1.0976
Epoch 300, Loss: 0.5064
Epoch 400, Loss: 0.4815
Epoch 500, Loss: 0.4743
Epoch 600, Loss: 0.4709
Epoch 700, Loss: 0.4690
Epoch 800, Loss: 0.4677
Epoch 900, Loss: 0.4668
